# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 11:01AM,247378,10,Enova-10707,Emerginnova,Released
1,Sep 27 2022 11:00AM,247377,16,8577323,Sartorius Bioprocess Solutions,Released
2,Sep 27 2022 10:59AM,247369,20,ALUR154340149,Alumier Labs Inc.,Released
3,Sep 27 2022 10:59AM,247369,20,ALUR378664794,Alumier Labs Inc.,Released
4,Sep 27 2022 10:59AM,247369,20,ALU0008018,Alumier Labs Inc.,Released
5,Sep 27 2022 10:58AM,247376,16,8560855,Sartorius Bioprocess Solutions,Released
6,Sep 27 2022 10:56AM,247375,16,8576764,Sartorius Lab Products and Service,Released
7,Sep 27 2022 10:56AM,247375,16,8576776,Sartorius Lab Products and Service,Released
8,Sep 27 2022 10:56AM,247375,16,8576777,Sartorius Lab Products and Service,Released
9,Sep 27 2022 10:56AM,247375,16,8576779,Sartorius Lab Products and Service,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,247374,Released,1
34,247375,Released,6
35,247376,Released,1
36,247377,Released,1
37,247378,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247374,NaN,NaN,1.0
247375,NaN,NaN,6.0
247376,NaN,NaN,1.0
247377,NaN,NaN,1.0
247378,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247201,0.0,0.0,1.0
247202,0.0,0.0,1.0
247205,0.0,0.0,1.0
247250,0.0,1.0,0.0
247258,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247201,0,0,1
247202,0,0,1
247205,0,0,1
247250,0,1,0
247258,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247201,0,0,1
1,247202,0,0,1
2,247205,0,0,1
3,247250,0,1,0
4,247258,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247201,,,1
1,247202,,,1
2,247205,,,1
3,247250,,1,
4,247258,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 11:01AM,247378,10,Emerginnova
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.
5,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions
6,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service
12,Sep 27 2022 10:55AM,247374,19,"Methapharm, Inc."
13,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.
20,Sep 27 2022 10:40AM,247372,19,ACG North America LLC
21,Sep 27 2022 10:39AM,247368,10,Bio-PRF
34,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 11:01AM,247378,10,Emerginnova,,,1
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions,,,1
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.,,,3
3,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions,,,1
4,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service,,,6
5,Sep 27 2022 10:55AM,247374,19,"Methapharm, Inc.",,,1
6,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,,,7
7,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,,,1
8,Sep 27 2022 10:39AM,247368,10,Bio-PRF,,,13
9,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 11:01AM,247378,10,Emerginnova,1,,
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions,1,,
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.,3,,
3,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions,1,,
4,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service,6,,
5,Sep 27 2022 10:55AM,247374,19,"Methapharm, Inc.",1,,
6,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,7,,
7,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,1,,
8,Sep 27 2022 10:39AM,247368,10,Bio-PRF,13,,
9,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 11:01AM,247378,10,Emerginnova,1,,
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions,1,,
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.,3,,
3,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions,1,,
4,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 11:01AM,247378,10,Emerginnova,1.0,NaN,NaN
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.,3.0,NaN,NaN
3,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 11:01AM,247378,10,Emerginnova,1.0,0.0,0.0
1,Sep 27 2022 11:00AM,247377,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Sep 27 2022 10:59AM,247369,20,Alumier Labs Inc.,3.0,0.0,0.0
3,Sep 27 2022 10:58AM,247376,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Sep 27 2022 10:56AM,247375,16,Sartorius Lab Products and Service,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3462935,104.0,3.0,1.0
15,494707,74.0,32.0,0.0
16,742128,8.0,0.0,0.0
19,1484004,40.0,4.0,0.0
20,741924,21.0,16.0,18.0
21,988858,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3462935,104.0,3.0,1.0
1,15,494707,74.0,32.0,0.0
2,16,742128,8.0,0.0,0.0
3,19,1484004,40.0,4.0,0.0
4,20,741924,21.0,16.0,18.0
5,21,988858,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,3.0,1.0
1,15,74.0,32.0,0.0
2,16,8.0,0.0,0.0
3,19,40.0,4.0,0.0
4,20,21.0,16.0,18.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,15,Released,74.0
2,16,Released,8.0
3,19,Released,40.0
4,20,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,18.0,0.0
Executing,3.0,32.0,0.0,4.0,16.0,1.0
Released,104.0,74.0,8.0,40.0,21.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,18.0,0.0
1,Executing,3.0,32.0,0.0,4.0,16.0,1.0
2,Released,104.0,74.0,8.0,40.0,21.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,18.0,0.0
1,Executing,3.0,32.0,0.0,4.0,16.0,1.0
2,Released,104.0,74.0,8.0,40.0,21.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()